This notebook will be used for the IBM DS Capstone project.

In [1]:
import pandas as pd
import numpy as np
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  openssl                                 1.1.1g-h516909a_0 --> 1.1.1g-h516909a_1



openssl-1.1.1g       | 2.1 MB    | #####

In [3]:
with open('schools.json') as json_data:
    schools_data = json.load(json_data)

In [4]:
schools_data[0]

{'datasetid': 'schools',
 'recordid': '72debd5c182c76d963b720b209a607a6a2382e0f',
 'fields': {'school_category': 'Public School',
  'geo_local_area': 'Strathcona',
  'geom': {'type': 'Point',
   'coordinates': [-123.08031429999998, 49.27859426906773]},
  'school_name': 'Admiral Seymour Elementary',
  'address': '1130 Keefer St'},
 'record_timestamp': '2019-06-01T04:01:22.964-07:00'}

In [5]:
# define the dataframe columns
column_names = ['area', 'school_name', 'address', 'school_type', 'Latitude', 'Longitude'] 

# instantiate the dataframe
schools = pd.DataFrame(columns=column_names)

In [6]:
for data in schools_data:
    area_ = data.get('fields').get('geo_local_area')
    school_name_=data.get('fields').get('school_name')
    address_=data.get('fields').get('address')
    school_type_=data.get('fields').get('school_category')
    Latitude_=data.get('fields').get('geom').get('coordinates')[1]
    Longitude_=data.get('fields').get('geom').get('coordinates')[0]
    
    schools = schools.append({'area': area_, 'school_name': school_name_, 'address': address_, 'school_type':school_type_, 
                              'Latitude':Latitude_, 'Longitude':Longitude_}, ignore_index=True)       
    

In [7]:
schools.head()

AttributeError: 'NoneType' object has no attribute 'items'

                 area                           school_name  \
0          Strathcona            Admiral Seymour Elementary   
1            Downtown               Anchor Point Montessori   
2        South Cambie  BCCH Eating Disorders School Program   
3            Fairview              Blessed Sacrament School   
4  Grandview-Woodland        Britannia Community Elementary   

             address         school_type   Latitude   Longitude  
0     1130 Keefer St       Public School  49.278594 -123.080314  
1     1351 Hornby St  Independent School  49.277062 -123.130919  
2     5025 Willow St       Public School  49.239631 -123.125791  
3    3020 Heather St  Independent School  49.258338 -123.119391  
4  1110 Cotton Drive       Public School  49.274512 -123.071379  

In [8]:
address = 'Vancouver, BC'

geolocator = Nominatim(user_agent="Van_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vancouver are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vancouver are 49.2608724, -123.1139529.


In [9]:
# create map of Vancouver using latitude and longitude values
map_van = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(schools['Latitude'], schools['Longitude'], schools['school_name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_van)  
    
map_van

In [10]:
CLIENT_ID = 'GAEXUWPZOQTIMXLACOEOLTR4Y5RLBSWJY0EUJF4FZ3C1CMNO' # your Foursquare ID
CLIENT_SECRET = 'G3TQZYJ2VTEJSRAXVCWESG5JRMMOUYD1NIYPN3KWC2KSOM3P' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GAEXUWPZOQTIMXLACOEOLTR4Y5RLBSWJY0EUJF4FZ3C1CMNO
CLIENT_SECRET:G3TQZYJ2VTEJSRAXVCWESG5JRMMOUYD1NIYPN3KWC2KSOM3P


In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['school_name', 
                  'School Latitude', 
                  'School Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
LIMIT = 100

van_venues = getNearbyVenues(names=schools['school_name'],
                                   latitudes=schools['Latitude'],
                                   longitudes=schools['Longitude']
                                  )


Admiral Seymour Elementary
Anchor Point Montessori
BCCH Eating Disorders School Program
Blessed Sacrament School
Britannia Community Elementary
Century High School
Champlain Heights Annex
Champlain Heights Annex StrongStart Cntr
Charles Dickens Annex
Chief Maquinna Annex
Crofton House
Crosstown Elementary
David Livingstone Elementary
Dr George M Weir Elementary
Dr H N MacCorkindale Elementary
Edith Cavell Elementary
Florence Nightingale Elementary
G F Strong School Program
General Brock Elementary
General Wolfe Elementary
Grandview Elementary
Hastings Community Elementary
Ivy Montessori School
John Henderson Annex
Kerrisdale Elementary
Killarney Secondary
Laura Secord Elementary
Lord Roberts Elementary
Lord Selkirk Elementary
Lord Strathcona Community Elementary
Lord Tennyson Elementary
MacDonald StrongStart Centre
Maple Grove Elementary
Mount Pleasant Elementary
Norma Rose Point School
Pacific Torah Institute
Pattison High School
Pattison School
Point Grey Secondary
Prince of Wales Se

In [13]:
van_venues.head(10)

AttributeError: 'NoneType' object has no attribute 'items'

                  school_name  School Latitude  School Longitude  \
0  Admiral Seymour Elementary        49.278594       -123.080314   
1  Admiral Seymour Elementary        49.278594       -123.080314   
2  Admiral Seymour Elementary        49.278594       -123.080314   
3  Admiral Seymour Elementary        49.278594       -123.080314   
4  Admiral Seymour Elementary        49.278594       -123.080314   
5  Admiral Seymour Elementary        49.278594       -123.080314   
6  Admiral Seymour Elementary        49.278594       -123.080314   
7  Admiral Seymour Elementary        49.278594       -123.080314   
8  Admiral Seymour Elementary        49.278594       -123.080314   
9  Admiral Seymour Elementary        49.278594       -123.080314   

                     Venue  Venue Latitude  Venue Longitude  \
0           La Casa Gelato       49.276923      -123.081549   
1                   Fujiya       49.276623      -123.077164   
2  Off The Rail Brewing Co       49.277690      -123.076102   

In [14]:
van_venues.shape

(4274, 7)

In [15]:
df=van_venues.groupby(['school_name', 'School Latitude', 'School Longitude']).count().reset_index()
df=df.drop(['Venue Latitude', 'Venue Longitude', 'Venue Category'], axis = 1)

In [16]:
df=df.rename(columns={"Venue": "Venue_Count"})
df.head()

AttributeError: 'NoneType' object has no attribute 'items'

                          school_name  School Latitude  School Longitude  \
0          Admiral Seymour Elementary        49.278594       -123.080314   
1  Admiral Seymour StrongStart Centre        49.278594       -123.080314   
2                   Alexander Academy        49.285001       -123.114010   
3             Anchor Point Montessori        49.277062       -123.130919   
4     BC Children's Adol. Psych. Unit        49.239631       -123.125791   

   Venue_Count  
0           19  
1           19  
2          100  
3           58  
4            5  

In [17]:
schools.shape

(194, 6)

In [18]:
df.shape

(192, 4)

In [19]:
df_cluster=df.drop(['school_name'], axis = 1)
df_cluster.columns

Index(['School Latitude', 'School Longitude', 'Venue_Count'], dtype='object')

In [20]:
from sklearn.preprocessing import StandardScaler

X = df_cluster.values[:,1:]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset

array([[ 0.40769432, -0.1426882 ],
       [ 0.40769432, -0.1426882 ],
       [-0.28863982,  3.40217913],
       [-0.6380735 ,  1.56409977],
       [-0.5321018 , -0.75538132],
       [-0.5321018 , -0.75538132],
       [-0.5321018 , -0.75538132],
       [-1.70058542,  1.08269803],
       [-0.39984182,  0.5575325 ],
       [ 0.59235064,  1.91421012],
       [ 0.58110237,  1.78291874],
       [-0.88902405, -0.75538132],
       [ 1.3270025 , -0.23021579],
       [ 1.3270025 , -0.23021579],
       [-1.51921394, -0.62408994],
       [-0.93556273,  1.17022562],
       [ 1.43610292, -0.66785373],
       [ 1.43610292, -0.66785373],
       [ 1.49399389, -0.88667271],
       [ 0.35817967,  0.29494974],
       [ 0.36478595, -0.44903476],
       [ 1.13504021, -0.1426882 ],
       [ 1.02196398, -0.36150718],
       [ 1.02196398, -0.36150718],
       [ 1.06826293, -0.71161753],
       [ 1.39966329,  0.42624112],
       [ 1.39966329,  0.42624112],
       [ 0.10260217, -0.44903476],
       [-0.83112192,

In [21]:
num_clusters = 5

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

print(labels)

[0 0 3 1 2 2 2 1 1 1 1 2 0 0 4 1 0 0 0 0 0 0 0 0 0 0 0 2 3 0 4 1 1 1 1 0 0
 0 2 0 0 4 4 1 1 2 2 2 4 1 1 4 2 0 0 2 1 1 0 0 1 0 3 4 4 2 2 2 2 0 2 4 4 0
 1 3 4 2 0 2 0 4 4 0 0 3 3 0 0 0 2 1 1 4 4 4 1 1 0 4 0 0 1 1 4 2 2 3 3 3 2
 1 4 0 0 0 4 4 4 1 4 0 2 0 2 2 2 2 0 1 0 0 0 0 0 2 0 0 0 0 2 2 1 2 2 2 1 2
 1 1 0 4 3 4 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 4 0 3 0 2 2 0 2 2 2 2 0 2 2 0 0
 0 4 2 0 0 4 2]


In [22]:
df_cluster["Labels"] = labels
df_cluster.head(5)

AttributeError: 'NoneType' object has no attribute 'items'

   School Latitude  School Longitude  Venue_Count  Labels
0        49.278594       -123.080314           19       0
1        49.278594       -123.080314           19       0
2        49.285001       -123.114010          100       3
3        49.277062       -123.130919           58       1
4        49.239631       -123.125791            5       2

In [23]:
df_cluster.groupby('Labels').mean()

AttributeError: 'NoneType' object has no attribute 'items'

        School Latitude  School Longitude  Venue_Count
Labels                                                
0             49.249964       -123.051947    14.346667
1             49.259896       -123.110636    45.705882
2             49.234794       -123.116393     9.704545
3             49.275732       -123.135124    89.545455
4             49.250496       -123.176532     8.285714

In [25]:
df_cluster_merged = pd.merge(df_cluster, df, on=['School Latitude', 'School Longitude']) 
df_cluster_merged.head()

AttributeError: 'NoneType' object has no attribute 'items'

   School Latitude  School Longitude  Venue_Count_x  Labels  \
0        49.278594       -123.080314             19       0   
1        49.278594       -123.080314             19       0   
2        49.278594       -123.080314             19       0   
3        49.278594       -123.080314             19       0   
4        49.285001       -123.114010            100       3   

                          school_name  Venue_Count_y  
0          Admiral Seymour Elementary             19  
1  Admiral Seymour StrongStart Centre             19  
2          Admiral Seymour Elementary             19  
3  Admiral Seymour StrongStart Centre             19  
4                   Alexander Academy            100  

In [27]:
df_cluster_merged=df_cluster_merged.drop(['Venue_Count_y'], axis = 1)
df_cluster_merged=df_cluster_merged.rename(columns={"Venue_Count_x": "Venue_Count"})
df_cluster_merged.head()

AttributeError: 'NoneType' object has no attribute 'items'

   School Latitude  School Longitude  Venue_Count  Labels  \
0        49.278594       -123.080314           19       0   
1        49.278594       -123.080314           19       0   
2        49.278594       -123.080314           19       0   
3        49.278594       -123.080314           19       0   
4        49.285001       -123.114010          100       3   

                          school_name  
0          Admiral Seymour Elementary  
1  Admiral Seymour StrongStart Centre  
2          Admiral Seymour Elementary  
3  Admiral Seymour StrongStart Centre  
4                   Alexander Academy  

In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, ven, cluster in zip(df_cluster_merged['School Latitude'], df_cluster_merged['School Longitude'], df_cluster_merged['school_name'], 
    df_cluster_merged['Venue_Count'], df_cluster_merged['Labels']):
    label = folium.Popup(str(poi) + 'C' + str(cluster) + 'V' + str(ven), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [19]:
result=pd.merge(schools, df, on='school_name')

In [20]:
result.columns

Index(['area', 'school_name', 'address', 'school_type', 'Latitude',
       'Longitude', 'School Latitude', 'School Longitude', 'Venue_Count'],
      dtype='object')

In [21]:
result=result.drop(['address', 'Latitude', 'Longitude'], axis = 1)
result.columns

Index(['area', 'school_name', 'school_type', 'School Latitude',
       'School Longitude', 'Venue_Count'],
      dtype='object')

In [22]:
result=result.sort_values(by=['Venue_Count'], ascending=False)

In [27]:
result.head()

AttributeError: 'NoneType' object has no attribute 'items'

         area            school_name         school_type  School Latitude  \
156  West End     Lord Roberts Annex       Public School              100   
134  Downtown      Alexander Academy  Independent School              100   
37   Downtown        Pattison School  Independent School              100   
36   Downtown   Pattison High School  Independent School              100   
154  West End  King George Secondary       Public School              100   

     School Longitude  Venue_Count  
156               100          100  
134               100          100  
37                100          100  
36                100          100  
154               100          100  

In [24]:
result_area=result.groupby(['area','school_name','school_type']).sum()
result_area

AttributeError: 'NoneType' object has no attribute 'items'

                                                                                      School Latitude  \
area                     school_name                              school_type                           
Arbutus-Ridge            Carnarvon Community Elementary           Public School                     8   
                         Ivy Montessori School                    Independent School               17   
                         Prince of Wales Secondary                Public School                     4   
                         Trafalgar Elementary                     Public School                     4   
Downtown                 Alexander Academy                        Independent School              100   
                         Anchor Point Montessori                  Independent School               58   
                         Crosstown Elementary                     Public School                    56   
                         Elsie Roy Elementary          

In [25]:
result_area_50=result_area[result_area['Venue_Count'] >= 50]

In [26]:
result_area_50

AttributeError: 'NoneType' object has no attribute 'items'

                                                                      School Latitude  \
area               school_name                    school_type                           
Downtown           Alexander Academy              Independent School              100   
                   Anchor Point Montessori        Independent School               58   
                   Crosstown Elementary           Public School                    56   
                   Pattison High School           Independent School              100   
                   Pattison School                Independent School              100   
Fairview           Continuing Ed SD 39            Public School                    85   
                   Peak House                     Public School                    85   
                   Vancouver Alternate            Public School                    85   
Grandview-Woodland Britannia Community Elementary Public School                    66   
                   Br